# Process MODIS Vegetation Indices

### Prepare Workspace

In [190]:
# Import system libraries
import os
import sys

# Import data manipulation libraries
import pandas as pd
import numpy as np

# Import geospatial libraries
from shapely.geometry import Polygon, mapping
import geopandas as gpd
import xarray as xr
import rioxarray
import rasterio.mask

# Import API libraries
import pystac_client
import planetary_computer
import odc.stac
import rich.table

# Set working directory
os.chdir('/Users/jessicarapson/Documents/GitHub/water-supply-forecast')

### Load Data from API

In [198]:
# Call API
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Load in site geospatial data
gdf_sites = gpd.read_file('assets/data/geospatial.gpkg')

# Initialize an empty list to store catchment bounding boxes
site_bboxes = []

# Iterate through each polygon (catchment) in the GeoDataFrame
for index, row in gdf_sites.iterrows():
    # Get the bounding box for each polygon
    bbox = row.geometry.bounds  # Extract the bounding box as (minx, miny, maxx, maxy)
    site_bboxes.append(bbox)  # Append the bounding box to the list
    
# Initialise dataframes to store extracted information
df_jan = pd.DataFrame(
    {'site_id': gdf_sites['site_id'],'year': np.nan,'month': np.nan,
     'max_veg':np.nan,'min_veg': np.nan,
     'mean_veg': np.nan,'med_veg': np.nan,'percent_no_veg': np.nan,
     'percent_bare_soil': np.nan,'percent_any_veg': np.nan,
     'percent_sparse_veg': np.nan,'percent_dense_veg': np.nan})
df_apr = df_jan.copy()
df_jul = df_jan.copy()
df_oct = df_jan.copy()

# Create dataframe of months
months_dataframes = {'January': df_jan, 'April': df_apr, 'July': df_jul, 'October': df_oct}

# Collect data for each year
df_all = []

# Loop through years
for year in range(2020,2023 + 1):

    # Loop through catchment polygons
    for i in range(0,len(gdf_sites)):
        print("Processing MODIS for:", str(year), gdf_sites.iloc[i]['site_id'],
          f"({i + 1}/{len(gdf_sites)})")

        # Load the catchment polygon
        catchment_polygon = gdf_sites.geometry.iloc[i]

        # Select catchment bounding box
        bbox = site_bboxes[i]

        # Select dates
        months = {
            "January": "01",
            "April": "04",
            "July": "07",
            "October": "10",
        }
        items_season = dict()

        # Search using bounding box coordinates
        items_full = []
        for name, number in months.items():
            datetime = f"{year}-{number}"
            search = catalog.search(
                collections=["modis-13A1-061"],
                bbox=bbox,
                datetime=datetime,
            )
            items_season[name] = search.item_collection()[0]
            items = list(search.item_collection())
            items_trunc = items[0:len(set([tuple(x.bbox) for x in items]))]
            items_full += items_trunc

        # Load and merge data into xarray
        datasets = []
        item_num = 0
        for item in items_full:
            item_num += 1
            print("Processing item:", f"{item_num}/{len(items_full)}")
            data = odc.stac.load(
                [item],
                crs="EPSG:32610",
                bands="500m_16_days_NDVI",
                resolution=500,
                bbox=bbox)
            try:
                # Clip data for each catchment polygon
                data_clipped = data.rio.clip(
                    gdf_sites.geometry.apply(mapping)[[i]], gdf_sites.crs)
                # Mask areas outside the catchment polygon with NaN
                mask = ~data_clipped.isnull()
                data_clipped = data_clipped.where(mask)
                # Append data
                datasets.append(data_clipped)
            except rioxarray.exceptions.NoDataInBounds:
                continue  # Skip to the next item if no data is found

        # Merge the datasets using xarray
        merged_data = xr.concat(datasets, dim='item_index').mean(dim='item_index')

        # Convert data to raster
        raster = items_season["January"].assets["500m_16_days_NDVI"].extra_fields["raster:bands"]
        data = merged_data["500m_16_days_NDVI"] * raster[0]["scale"]

        # Clip data again so outside values are NaN
        data_clipped = data.rio.clip(gdf_sites.geometry.apply(mapping)[[i]], gdf_sites.crs)

        # Iterate through each month DataFrame
        for month, df in months_dataframes.items():

            # Label year
            df.at[i,'year'] = year

            # Select season
            data_season = data_clipped.sel(time=data['time.month'] == int(months[month]))[0]

             # Label month
            df.at[i,'month'] = np.datetime_as_string(data_season.time.values, unit='M')[-2:]

            # Extract mean, minimum, and average vegetation for catchment
            df.at[i,'max_veg'] = np.nanmax(data_season)
            df.at[i,'min_veg'] = np.nanmin(data_season)
            df.at[i,'mean_veg'] = np.nanmean(data_season)
            df.at[i,'med_veg'] = np.nanmedian(data_season)

            # Calculate percent vegetation exceeding various thresholds
            total_cells = np.sum(~np.isnan(data_season.values))
            df.at[i,'percent_no_veg'] = np.sum(data_season.values < 0) / total_cells
            df.at[i,'percent_bare_soil'] = np.sum(
                (data_season.values < 0.01) & (data_season.values > -0.01)) / total_cells
            df.at[i,'percent_any_veg'] = np.sum(data_season.values > 0) / total_cells
            df.at[i,'percent_sparse_veg'] = np.sum(
                (data_season.values > 0.1) & (data_season.values < 0.5)) / total_cells
            df.at[i,'percent_dense_veg'] = np.sum(data_season.values > 0.6) / total_cells

             # Export clipped and compressed raster file
            data_season.rio.to_raster('assets/data/modis/' + gdf_sites.iloc[i].site_id
                                      + '_' + str(year) + '_' + month + '_nvdi.tif')

    # Combine month dataframes
    df_year = pd.concat([df_jan, df_apr, df_jul, df_oct], ignore_index=True)
    df_all.append(df_year)
    print('\n###################################################\n')

# Export dataframe
df = pd.concat(df_all, ignore_index=True)
df.to_csv('assets/data/modis/modis_summary.csv', index=False)

Processing MODIS for: 2020 hungry_horse_reservoir_inflow (1/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8


/var/folders/ws/9hc4t42j3ls9vb_fw81ddmhw0000gn/T/ipykernel_88373/397287414.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i,'month'] = np.datetime_as_string(data_season.time.values, unit='M')[-2:]
/var/folders/ws/9hc4t42j3ls9vb_fw81ddmhw0000gn/T/ipykernel_88373/397287414.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '04' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i,'month'] = np.datetime_as_string(data_season.time.values, unit='M')[-2:]
/var/folders/ws/9hc4t42j3ls9vb_fw81ddmhw0000gn/T/ipykernel_88373/397287414.py:117: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '07' has dtype incompatible with float64, plea

Processing MODIS for: 2020 snake_r_nr_heise (2/26)
Processing item: 1/16
Processing item: 2/16
Processing item: 3/16
Processing item: 4/16
Processing item: 5/16
Processing item: 6/16
Processing item: 7/16
Processing item: 8/16
Processing item: 9/16
Processing item: 10/16
Processing item: 11/16
Processing item: 12/16
Processing item: 13/16
Processing item: 14/16
Processing item: 15/16
Processing item: 16/16
Processing MODIS for: 2020 pueblo_reservoir_inflow (3/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2020 sweetwater_r_nr_alcova (4/26)
Processing item: 1/16
Processing item: 2/16
Processing item: 3/16
Processing item: 4/16
Processing item: 5/16
Processing item: 6/16
Processing item: 7/16
Processing item: 8/16
Processing item: 9/16
Processing item: 10/16
Processing item: 11/16
Processing item: 12/16
Processing item: 13/16
Processing item: 

Processing item: 15/16
Processing item: 16/16
Processing MODIS for: 2021 pueblo_reservoir_inflow (3/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2021 sweetwater_r_nr_alcova (4/26)
Processing item: 1/16
Processing item: 2/16
Processing item: 3/16
Processing item: 4/16
Processing item: 5/16
Processing item: 6/16
Processing item: 7/16
Processing item: 8/16
Processing item: 9/16
Processing item: 10/16
Processing item: 11/16
Processing item: 12/16
Processing item: 13/16
Processing item: 14/16
Processing item: 15/16
Processing item: 16/16
Processing MODIS for: 2021 missouri_r_at_toston (5/26)
Processing item: 1/16
Processing item: 2/16
Processing item: 3/16
Processing item: 4/16
Processing item: 5/16
Processing item: 6/16
Processing item: 7/16
Processing item: 8/16
Processing item: 9/16
Processing item: 10/16
Processing item: 11/16
Processing it

Processing item: 6/14
Processing item: 7/14
Processing item: 8/14
Processing item: 9/14
Processing item: 10/14
Processing item: 11/14
Processing item: 12/14
Processing item: 13/14
Processing item: 14/14
Processing MODIS for: 2022 missouri_r_at_toston (5/26)
Processing item: 1/14
Processing item: 2/14
Processing item: 3/14
Processing item: 4/14
Processing item: 5/14
Processing item: 6/14
Processing item: 7/14
Processing item: 8/14
Processing item: 9/14
Processing item: 10/14
Processing item: 11/14
Processing item: 12/14
Processing item: 13/14
Processing item: 14/14
Processing MODIS for: 2022 animas_r_at_durango (6/26)
Processing item: 1/8
Processing item: 2/8
Processing item: 3/8
Processing item: 4/8
Processing item: 5/8
Processing item: 6/8
Processing item: 7/8
Processing item: 8/8
Processing MODIS for: 2022 yampa_r_nr_maybell (7/26)
Processing item: 1/22
Processing item: 2/22
Processing item: 3/22
Processing item: 4/22
Processing item: 5/22
Processing item: 6/22
Processing item: 7/22


Processing item: 18/18
Processing MODIS for: 2023 libby_reservoir_inflow (8/26)
Processing item: 1/18
Processing item: 2/18
Processing item: 3/18
Processing item: 4/18
Processing item: 5/18
Processing item: 6/18
Processing item: 7/18
Processing item: 8/18
Processing item: 9/18
Processing item: 10/18
Processing item: 11/18
Processing item: 12/18
Processing item: 13/18
Processing item: 14/18
Processing item: 15/18
Processing item: 16/18
Processing item: 17/18
Processing item: 18/18
Processing MODIS for: 2023 boise_r_nr_boise (9/26)
Processing item: 1/6
Processing item: 2/6
Processing item: 3/6
Processing item: 4/6
Processing item: 5/6
Processing item: 6/6
Processing MODIS for: 2023 green_r_bl_howard_a_hanson_dam (10/26)
Processing item: 1/6
Processing item: 2/6
Processing item: 3/6
Processing item: 4/6
Processing item: 5/6
Processing item: 6/6
Processing MODIS for: 2023 taylor_park_reservoir_inflow (11/26)
Processing item: 1/6
Processing item: 2/6
Processing item: 3/6
Processing item: 4/

IndexError: list index out of range